In [1]:
# 0 Housekeeping. Clear variable space
######################################
from IPython import get_ipython  

ipython = get_ipython()
ipython.magic("reset -f")
ipython = get_ipython()
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")

In [2]:
# 1 Import Libraries and Set Global Parameters
##############################################
# 1.1 Import Python Libraries
#############################
from datetime import datetime
print("Run Section 1 Import Libraries and Set Global Parameters...")
begin_time = datetime.now()

import os, sys
import pandas as pd
import geopandas as gpd
import pyarrow.parquet as pq

Run Section 1 Import Libraries and Set Global Parameters...


In [3]:
# 1.2 Set Global Parameters
###########################

if os.getlogin() == "WylieTimmerman":
    path_working = r"C:\OD\OneDrive - Foursquare ITP\Projects\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(r"C:\OD\OneDrive - Foursquare ITP\Projects\WMATA_AVL")
    path_sp = r"C:\Users\WylieTimmerman\Documents\projects_local\wmata_avl_local"
    path_source_data = os.path.join(path_sp,"data","00-raw")
    path_processed_data = os.path.join(path_sp, "data","02-processed")

elif os.getlogin() == "abibeka":
    path_working = r"C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\Github\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(path_working)
    path_source_data = r"C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\WMATA-AVL\Data"
    path_processed_data = os.path.join(path_source_data, "ProcessedData")
    
elif os.getlogin() == "E048374":
    # Working Paths
    path_working = r"C:\Users\E048374\OneDrive - WMATA\rawnav_rachel_fork\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(r"C:\Users\E048374\OneDrive - WMATA\rawnav_rachel_fork\WMATA_AVL")
    path_source_data = r"\\l-600730\RawNavArchive"
    path_sp = r"C:\Users\E048374\Documents\RawNav"
    path_processed_data = os.path.join(path_sp, "data", "02-processed")
    path_segments = path_processed_data

else:
    raise FileNotFoundError("Define the path_working, path_source_data, gtfs_dir, \
                            ZippedFilesloc, and path_processed_data in a new elif block")

In [4]:
# Globals
# Queue Jump Routes
q_jump_route_list = ['52']
analysis_routes = q_jump_route_list
# EPSG code for WMATA-area work
wmata_crs = 2248
     
# 1.3 Import User-Defined Package
#################################
import wmatarawnav as wr

In [5]:
seg_pattern = pd.read_csv(os.path.join(path_segments,"test_seg_pattern_5201.csv"),
                         dtype={'route':str, 'PATTERN_ID':str})
seg_pattern

,seg_name_id,PATTERN_ID,route,pattern,direction,from_geoid,from_stop_seq,to_stop_seq,to_geoid,stop_id
0,16203-16662,5201,52,1,SOUTH,16203,16,17,16662,19066


In [6]:
xwalk_seg_pattern_stop = seg_pattern[['route','pattern','seg_name_id','stop_id']].copy()
xwalk_seg_pattern_stop

,route,pattern,seg_name_id,stop_id
0,52,1,16203-16662,19066


In [7]:
# 2. Decompose Travel Time
##########################

# 2.0 Setup exports
###################
freeflow_list = []
stop_area_decomp_list = []
traveltime_decomp_list = []

path_exports = (
    os.path.join(
        path_processed_data,
        "exports_{}".format(datetime.now().strftime("%Y%m%d"))
    )
)
if not os.path.isdir(path_exports):
    os.mkdir(path_exports)

In [8]:
# 1 segment
seg = xwalk_seg_pattern_stop.loc[:,'seg_name_id'].values[0]
seg

'16203-16662'

In [12]:
print('now on {}'.format(seg))
# 2.1. Read-in Data 
###################
# Reduce rawnav data to runs present in the summary file after filtering.

xwalk_seg_pattern_stop_fil = xwalk_seg_pattern_stop.query('seg_name_id == @seg')

seg_routes = list(xwalk_seg_pattern_stop_fil.route.drop_duplicates())

rawnav_dat = (
    wr.read_cleaned_rawnav(
       analysis_routes_ = seg_routes,
       path = os.path.join(path_processed_data, "rawnav_data.parquet")
    )
    .drop(columns=['blank', 'lat_raw', 'long_raw', 'sat_cnt'])
)

segment_summary = (
    pq.read_table(
        source = os.path.join(path_processed_data,"segment_summary.parquet"),
        filters = [['seg_name_id', "=", seg]],
        use_pandas_metadata = True
    )
    .to_pandas()
)

segment_summary_fil = (
    segment_summary
    .query('~(flag_too_far_any\
              | flag_wrong_order_any\
              | flag_too_long_odom\
              | flag_secs_total_mismatch\
              | flag_odom_total_mismatch)'
    )
)

now on 16203-16662


In [13]:
#first import stop_index -- then join seg_name_id
stop_index = ( pq.read_table(source=os.path.join(path_processed_data,"stop_index.parquet"),
                  filters=[[('route','=',route)] for route in seg_routes],
                    columns = [ 'route',
                                'pattern',
                                'stop_id',
                                'filename',
                                'index_run_start',
                                'index_loc',
                                'odom_ft',
                                'sec_past_st',
                                'geo_description'],
                  use_pandas_metadata = True
    )
    .to_pandas())

In [14]:
stop_index.head(3)

,route,pattern,stop_id,filename,index_run_start,index_loc,odom_ft,sec_past_st,geo_description
0,52,1,6192,rawnav07164191116.txt,544.0,544.0,0.0,0.0,COLORADO AVE NW + 14TH ST NW
1,52,1,25231,rawnav07164191116.txt,544.0,560.0,162.0,143.0,14TH ST + JEFFERSON DR
2,52,1,21857,rawnav07164191116.txt,544.0,578.0,596.0,162.0,14TH ST + INGRAHAM ST


In [36]:
# stop_index_w_segid = stop_index.merge(xwalk_seg_pattern_stop_in, how='left', on=['route','stop_id']).copy()

In [15]:
stop_index.loc[:,'pattern'] = stop_index.loc[:,'pattern'].astype('int32').copy()
stop_index.rename(columns={'odom_ft' : 'odom_ft_qj_stop'}, inplace=True)

In [16]:
# Filter Stop index to the relevant QJ stops
stop_index_fil = (
    stop_index
    .merge(xwalk_seg_pattern_stop_fil,
           on = ['route','pattern','stop_id'],
           how = 'inner')   
)

In [18]:
stop_index_fil.head(3)

,route,pattern,stop_id,filename,index_run_start,index_loc,odom_ft_qj_stop,sec_past_st,geo_description,seg_name_id
0,52,1,19066,rawnav07164191116.txt,544.0,990.0,11096.0,856.0,14TH ST + HARVARD ST,16203-16662
1,52,1,19066,rawnav07164191116.txt,6398.0,6997.0,11086.0,1304.0,14TH ST + HARVARD ST,16203-16662
2,52,1,19066,rawnav07164191116.txt,9762.0,10342.0,11170.0,1131.0,14TH ST + HARVARD ST,16203-16662


In [19]:
segment_summary_fil.iloc[0]

filename                                                     rawnav07164191116.txt
index_run_start                                                                544
pattern                                                                          1
start_date_time                                                2019-11-15 05:05:00
flag_too_far_any                                                             False
flag_wrong_order_any                                                         False
flag_too_long_odom                                                           False
flag_secs_total_mismatch                                                     False
flag_odom_total_mismatch                                                     False
start_odom_ft_segment                                                        10363
end_odom_ft_segment                                                          11208
trip_dist_mi_odom_and_segment                                                 0.16
star

In [20]:
# 2.2 Run Decomposition Functions
#################################
# These functions could be further nested within one another, but are kept separate to support
# easier review of intermediate outputs and to allow them to be used independently if needed.
# As a result, data is in some cases filtered several times, but the overall time loss is 
# small enough to be immaterial.

# Calculate Free Flow Travel Time through Entire Segment
segment_ff = (
    wr.decompose_segment_ff(
        rawnav_dat,
        segment_summary_fil,
        max_fps = 73.3
    )
    .assign(seg_name_id = seg)
)

freeflow_list.append(segment_ff)

In [21]:
freeflow_list

[      fps_next3        mph  seg_name_id
 0.01   0.433474   0.295483  16203-16662
 0.05   1.303030   0.888228  16203-16662
 0.10   4.333333   2.953874  16203-16662
 0.15   9.000000   6.134969  16203-16662
 0.25  14.333333   9.770507  16203-16662
 0.50  23.000000  15.678255  16203-16662
 0.75  30.333333  20.677119  16203-16662
 0.85  33.666667  22.949330  16203-16662
 0.90  36.000000  24.539877  16203-16662
 0.95  39.666667  27.039309  16203-16662
 0.99  47.500000  32.379005  16203-16662]

In [22]:
# Calculate Stop-Area Decomposition
stop_area_decomp = (
    wr.decompose_stop_area(
        rawnav_dat,
        segment_summary_fil,
        stop_index_fil
    )
    .assign(seg_name_id = seg)
)

stop_area_decomp_list.append(stop_area_decomp)

In [23]:
stop_area_decomp_list[0].iloc[0]

index                                          24
index_loc                                     983
lat                                       38.9272
long                                     -77.0325
heading                                       176
door_state                                      C
veh_state                                       M
odom_ft                                     10976
sec_past_st                                   850
stop_window                                      
row_before_apc                                  0
route_pattern                                5201
pattern                                         1
index_run_start                               544
index_run_end                                1808
filename                    rawnav07164191116.txt
start_date_time               2019-11-15 05:05:00
route                                          52
wday                                       Friday
odom_ft_next                                11005


In [24]:
segment_ff_val = (
    segment_ff
    .loc[0.95]
    .loc["fps_next3"]
)

# Run decomposition
traveltime_decomp = (
    wr.decompose_traveltime(
        rawnav_dat,
        segment_summary_fil,
        stop_area_decomp,
        segment_ff_val
    )
)
traveltime_decomp_list.append(traveltime_decomp)

In [25]:
traveltime_decomp_list[0].iloc[0]

filename             rawnav07164191116.txt
index_run_start                        544
seg_name_id                    16203-16662
route                                   52
pattern                                  1
                             ...          
lat_start                          38.9557
long_start                        -77.0332
lat_end                            38.8849
long_end                          -77.0213
odom_ft_seg_total                      845
Name: 0, Length: 61, dtype: object

In [26]:
# 3. Export
###########

# 3.1 Combine results by segment into tables   
########################################
freeflow = (
    pd.concat(freeflow_list)
    .rename_axis('ntile')
    .reset_index()
)

basic_decomp = (
    pd.concat(stop_area_decomp_list)
    .reset_index() 
)

traveltime_decomp = (
    pd.concat(traveltime_decomp_list)
    .reset_index()
)

In [27]:
# 3.2 Export Values
###################

freeflow.to_csv(os.path.join(path_exports,"freeflow.csv"))

basic_decomp.to_csv(os.path.join(path_exports,"basic_decomp.csv"))

traveltime_decomp.to_csv(os.path.join(path_exports,"traveltime_decomp.csv"))